In [1]:
!git clone https://github.com/Mike030668/Project_glass.git -q

In [2]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  import os
  os.chdir('/content/Project_glass')
  os.getcwd()

else:
  print('Not running on CoLab')

Running on CoLab


In [6]:
#from prepare_data.utils import csv2df, clean_dataset
from prepare_data.const_predprice import  TARGET_COLS, BASE_COLS, BATCH_SIZE, DEPTH, PREDICT_LAG
from prepare_data.constants import GLASS_COLS
import numpy as np # библиотека нампи

import tensorflow as tf           # библиотека машинного обучения

In [5]:
x_train_sc = np.load("./predprice/x_train_sc.npy")
x_val_sc = np.load("./predprice/x_val_sc.npy")
y_train_sc = np.load("./predprice/y_train_sc.npy")
y_val_sc = np.load("./predprice/y_val_sc.npy")

In [7]:
# Для генерации выборки временных рядов
TSG = tf.keras.preprocessing.sequence.TimeseriesGenerator

# Создаем генератор для обучения
train_datagen = TSG(x_train_sc, y_train_sc,          # В качестве параметров наши выборки
                    length = DEPTH,            # Анализируем по 21 прошедшим точкам
                    sampling_rate = 1,         # Для каждой точки
                    batch_size = BATCH_SIZE)   # Размер batch, который будем скармливать модели

# Создаем аналогичный генератор для валидации при обучении
val_datagen = TSG(x_val_sc, y_val_sc,                # В качестве параметров наши выборки
                    length = DEPTH,            # Анализируем по 21 прошедшим точкам
                    sampling_rate = 1,         # Для каждой точки
                    batch_size = BATCH_SIZE)   # Размер batch, который будем скармливать модели

print(train_datagen[0][0].shape,
      val_datagen[0][1].shape)

# Создадим генератор проверочной выборки, из которой потом вытащим xVal, yVal для проверки/оценки
DataGen = TSG(x_val_sc, y_val_sc,              # В качестве параметров наши выборки
              length = DEPTH,            # Анализируем по 21 прошедшим точкам
              sampling_rate = 1,         # Для каждой точки
              batch_size = len(x_val_sc))   # Размер batch будет равен длине нашей выборки


# сформируем сами массивы для проверки
XVAL, YVAL = [], []
for i in DataGen:
  XVAL.append(i[0])
  YVAL.append(i[1])

# так как это список объектов
# мы вытаскиваем нужный нам
XVAL = np.array(XVAL)[0]
YVAL = np.array(YVAL)[0]
print(XVAL.shape,YVAL.shape)

(64, 32, 53) (64, 5)
(530, 32, 53) (530, 5)


In [8]:
from scripts.make_models import baseline

model= baseline(input_shape = XVAL.shape[1:],
                    pred_num = PREDICT_LAG,
                    activ_out = "linear",
                    depth= DEPTH)


In [ ]:
# понижение шага
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 mode='min',
                                                 factor = 0.8,
                                                 patience = 3,
                                                 min_lr = 1e-9,
                                                 verbose = 1)

# остановка
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

path_model = '/content/Project_glass/predprice/model'

# Сохранение модели
checkpoint  = tf.keras.callbacks.ModelCheckpoint(
                        filepath = path_model,
                        #save_weights_only=True,
                        monitor='val_loss',
                        mode='min',
                        save_best_only=True
                        )
# оптимизатор
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-2)

# функция потерь
loss = tf.keras.losses.Huber() #.MSE #
model.compile(optimizer = optimizer, loss = loss, metrics=['mae', 'mape'])

In [ ]:
# обучение модели вашей
history = model.fit(train_datagen,                  # генератор данных для обучения
                    validation_data = val_datagen, # генератор данных для проверки
                    epochs = 2,
                    callbacks=[early_stopping, reduce_lr], #, checkpoint],
                    verbose = 1)

Epoch 1/2
34/34 [==============================] - 14s 136ms/step - loss: 0.1475 - mae: 0.4349 - mape: 496254.3438 - val_loss: 0.0177 - val_mae: 0.1563 - val_mape: 33.0704 - lr: 0.0100
Epoch 2/2
34/34 [==============================] - 1s 22ms/step - loss: 0.0401 - mae: 0.2221 - mape: 243577.9844 - val_loss: 0.0096 - val_mae: 0.1202 - val_mape: 28.0262 - lr: 0.0100
